In [1]:
!pip install fastapi uvicorn torch diffusers transformers accelerate pyngrok peft==0.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
  

In [2]:
# Import libraries
from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse
from pydantic import BaseModel
import torch
from diffusers import StableDiffusionPipeline
import uuid
import os
from google.colab import files
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from safetensors.torch import load_file
import logging
from peft import PeftModel
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
# Apply nest_asyncio to allow nested event loops (needed for Colab)
nest_asyncio.apply()


2025-04-22 23:25:35.935073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745364336.174130      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745364336.238021      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import FileResponse
from pydantic import BaseModel
import torch
import os
import uuid
import logging
from diffusers import StableDiffusionPipeline
from safetensors.torch import load_file
from peft import PeftModel

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ImageRequest(BaseModel):
    prompt: str
    negative_prompt: str = None
    num_inference_steps: int = 50
    guidance_scale: float = 7.5
    width: int = 512
    height: int = 512
    model: str = "finetuned"

# Create FastAPI app
app = FastAPI(title="Stable Diffusion API")

# Initialize stable diffusion model
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
# pipe_base = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)

# # Path to the LoRA adapter model
# unet_safetensor_path = "/kaggle/input/finetuned-v1"

# if os.path.exists(unet_safetensor_path):
#     try:
#         def null_safety(images, **kwargs):
#            return images, [False] * len(images)

#         pipe.safety_checker = null_safety
#         # Apply LoRA weights using PEFT
#         pipe.unet = PeftModel.from_pretrained(pipe.unet, unet_safetensor_path)
        
#         # Load the base model on GPU first
#         pipe = pipe.to("cuda")
#         print(f"Successfully loaded LoRA adapter from {unet_safetensor_path}")
#     except Exception as e:
#         print(f"Error loading LoRA adapter: {str(e)}")
#         print("Will use default weights instead")
#         # Make sure pipe is on GPU
#         pipe = pipe.to("cuda")
# else:
#     logger.warning(f"File not found at {unet_safetensor_path}, using default weights")
#     # Make sure pipe is on GPU
#     pipe = pipe.to("cuda")
#     # pipe_base = pipe_base.to("cuda") 
pipe = pipe.to("cuda")
# Create a directory to store generated images
os.makedirs("generated_images", exist_ok=True)

@app.post("/generate-image/")
async def generate_image(request: ImageRequest):
    try:
        # Generate the image
        
        image = pipe(
            prompt=request.prompt,
            negative_prompt= request.negative_prompt,
            num_inference_steps = request.num_inference_steps,
            guidance_scale = request.guidance_scale,
            width= request.width,
            height = request.height
        ).images[0]
       
        
        # Save the image with a unique filename
        filename = f"generated_images/{uuid.uuid4()}.png"
        image.save(filename)
        
        # Return the image file
        return FileResponse(filename, media_type="image/png")
    
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/")
async def root():
    return {"message": "Stable Diffusion API is running with LoRA adapter. Use /generate-image/ endpoint to create images."}

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [6]:
!ngrok config add-authtoken 2w6aON85CklbUifKabcl7dBj1pA_6pjZZJSFwy5JGhu5W8SmN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
# Set up ngrok for public URL (since Colab doesn't expose ports directly)
!pip install pyngrok
ngrok_tunnel = ngrok.connect(8000)
print(f"Public URL: {ngrok_tunnel.public_url}")

# Run the FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)

Public URL: https://b4b7-34-139-155-108.ngrok-free.app


INFO:     Started server process [31]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "POST /generate-image/ HTTP/1.1" 200 OK


  0%|          | 0/40 [00:00<?, ?it/s]

INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "POST /generate-image/ HTTP/1.1" 200 OK


  0%|          | 0/50 [00:00<?, ?it/s]

INFO:     2601:19b:280:f500:1188:ab0e:f924:a0a1:0 - "POST /generate-image/ HTTP/1.1" 200 OK
